In [103]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import os
from pydantic import BaseModel

import mimetypes

In [104]:
load_dotenv(override=True)

True

In [105]:
client = AsyncOpenAI(
    api_key=os.getenv("GEMINI_ACCESS_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=client
)

In [106]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [107]:
sales_agent_1=Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model=model
)

sales_agent_2=Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model=model
)

sales_agent_3=Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model=model
)

In [108]:
description="Write a cold sales email"

tool1=sales_agent_1.as_tool(tool_name="sales_agent_1",tool_description=description)
tool2=sales_agent_2.as_tool(tool_name="sales_agent_2",tool_description=description)
tool3=sales_agent_3.as_tool(tool_name="sales_agent_3",tool_description=description)

In [109]:
from email.message import EmailMessage
import ssl
import smtplib

@function_tool
def send_html_email(subject:str, html_body:str) ->Dict[str,str]:
    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = "himanshusinghvns343@gmail.com"
    msg["To"] = "himanshusinghvns343@gmail.com"
    msg.add_alternative(html_body, subtype="html")

    context = ssl.create_default_context()
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls(context=context)
        server.login(os.getenv('GMAIL_USER_ID'), os.getenv('GMAIL_API_KEY'))
        server.send_message(msg)

    return {"status": "success"}

In [110]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model=model)
subject_tool = subject_writer.as_tool(tool_name="subject_writer",tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body coverter",instructions=html_instructions,model=model)
html_tool=html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [111]:
email_tools=[subject_tool,html_tool,send_html_email]

In [112]:
instructions="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body. DO NOT include subject in body."

emailer_agent= Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model=model,
    handoff_description="Convert an email to HTML and send it"
)

In [113]:
tools=[tool1,tool2,tool3]
handoffs=[emailer_agent]

print(tools)

print(handoffs)

[FunctionTool(name='sales_agent_1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent_1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x138cfaac0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent_2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent_2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x138cf9a80>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent_3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'requir

In [114]:
sales_manager_instructions="""
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""

sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model=model
)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager,message)

<h3>Implementing Gaurdrails</h3>

In [115]:
class NameCheckOutput(BaseModel):
    is_name_in_message:bool
    name:str

guardrail_agent=Agent(
    name="Name check",
    instructions="Check if the user is including someones's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model=model
)

In [116]:
@input_guardrail
async def gaurdrail_against_name(ctx,agent,message):
    result = await Runner.run(guardrail_agent,message,context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name":result.final_output},tripwire_triggered=is_name_in_message)

In [118]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model=model,
    input_guardrails=[gaurdrail_against_name]
)

message="Send out a cold sales email addressed to Dear CEO from Head of Business Development"

# The below message will trigger thr gaurdrail because name is mentioned in the prompt.
#message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(careful_sales_manager,message)